In [1]:
import pandas as pd
import glob
file_paths = glob.glob(r'1984 - 2022 멜론 Top.100\\*.멜론TOP100가사.csv')
df_list = [pd.read_csv(file_path,index_col=0) for file_path in file_paths]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.rename(columns={'가사':'lyrics','제목':'title','장르':'genre'},inplace=True)
combined_df.drop_duplicates(ignore_index=True,inplace=True)
combined_df = combined_df[(combined_df['lyrics']!='Nan')&(combined_df['title']!='NaN') ]
combined_df.dropna(inplace=True)
combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n\n', '\n', regex=False)
df_1,df_2,df_3,df_4,df_5,df_6 = (combined_df.copy(deep=True) for _ in range(6))

In [2]:
#단어단위 뭉치 NL줄 단위\n",
def word_N_split(lyrics_row,l_num):
    l_num =l_num
    word_sum = 0
    lyrics_mung = []
    lyrics_word =[]
    for word in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]
        word_sum += len(word)
        lyrics_word.append(word)

        if word_sum>=l_num:
            lyrics_mung.append(lyrics_word)
            word_sum = 0
            lyrics_word=[]
    return [[word for phrase in sublist for word in phrase.split()] for sublist in lyrics_mung]

#한줄단위 뭉치 NL줄 단위
def line_N_split(lyrics_row,l_num):
    word_sum = 0
    lyrics_mung = []
    lyrics_word =[]
    for word in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]

        word_sum += len(word)
        lyrics_word.append(word)

        if word_sum>=l_num:
            lyrics_mung.append(lyrics_word)
            word_sum = 0
            lyrics_word=[]
    return lyrics_mung

def count_char(text):
    return sum(1 for char in text if char.isalpha())

In [3]:
# label 한syllable_strategy (필요없어짐)
#df['lyrics'].apply(lambda x: [line.split(' ') for line in x.split('\\n')] if pd.notnull(x) else x)

# label syllable_limit마다
n_ = 1
df_1['label'] = df_1['lyrics'].apply(lambda x: [' '.join(x.split('\n')[i:i+n_]).split(' ') for i in range(0, len(x.split('\n')), n_)])
df_1['chunk_strategy'] = 'word'
df_1['sampling_strategy'] ='one'
df_1['syllable_limit'] = 0
df_1['mungchi'] = df_1['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# label 전곡
df_2['label'] = df_2['lyrics'].apply(lambda x: [word for part in x.split('\n') for word in part.split(' ') if word])
df_2['chunk_strategy'] = 'word'
df_2['sampling_strategy'] ='total'
df_2['syllable_limit'] = 0
df_2['mungchi'] = df_2['label'].apply(lambda x: [count_char(sublist) for sublist in x])

l_num = 30
df_3['label'] = df_3['lyrics'].apply(lambda x: word_N_split(x,l_num))
df_3['chunk_strategy'] = 'word'
df_3['sampling_strategy'] ='L'
df_3['syllable_limit'] = l_num
df_3['mungchi'] = df_3['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label syllable_limit
n_ = 1
df_4['label'] = df_4['lyrics'].apply(lambda x: [x.strip().split('\n')[i:i+n_] for i in range(0, len(x.strip().split('\n')), n_)])
df_4['chunk_strategy'] = 'line'
df_4['sampling_strategy'] ='one'
df_4['syllable_limit'] = 0
df_4['mungchi'] = df_4['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label 전곡
df_5['label']= df_5['lyrics'].apply(lambda x: [x.strip().split('\n')])
df_5['chunk_strategy'] = 'line'
df_5['sampling_strategy'] ='total'
df_5['syllable_limit'] = 0
df_5['mungchi'] = df_5['label'].apply(lambda x: [[count_char(word.strip()) for word in sublist] for sublist in x])

df_6['label'] = df_6['lyrics'].apply(lambda x: line_N_split(x,l_num))
df_6['chunk_strategy'] = 'line'
df_6['sampling_strategy'] ='L'
df_6['syllable_limit'] = l_num
df_6['mungchi'] = df_6['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])

In [4]:
dataset = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], axis=0)
dataset = dataset[['title','lyrics','genre','mungchi','label','chunk_strategy','sampling_strategy','syllable_limit']]

In [18]:
# data 저장
dataset.to_csv('lyrics_dataset.csv',index_label=False)
dataset.reset_index().to_json('lyrics_dataset.json')

In [19]:
df = dataset
df.loc[(df['title']=='J에게') & (df['chunk_strategy']=='line') & (df['sampling_strategy']=='one')]['mungchi'][0]

[[15],
 [14],
 [14],
 [13],
 [17],
 [14],
 [14],
 [14],
 [14],
 [17],
 [14],
 [14],
 [14],
 [14],
 [7]]

In [22]:
df.loc[(df['title']=='J에게')]['mungchi']

0    [[1, 3, 3, 1, 2, 2, 3], [1, 3, 3, 2, 5], [1, 3...
0    [1, 3, 3, 1, 2, 2, 3, 1, 3, 3, 2, 5, 1, 3, 3, ...
0    [[1, 3, 3, 1, 2, 2, 3, 1, 3, 3, 2, 5], [1, 3, ...
0    [[15], [14], [14], [13], [17], [14], [14], [14...
0    [[15, 14, 14, 13, 17, 14, 14, 14, 14, 17, 14, ...
0    [[15, 14], [14, 13], [17, 14], [14, 14], [14, ...
Name: mungchi, dtype: object

In [39]:
# 리스트안에 빈 가사 찾기
idx = 0
for i in dataset.label:
    idx+=1
    for j in i:
        for x in j:
            if len(x)==0:
                print(idx,len(x),i)

AttributeError: 'DataFrame' object has no attribute 'label'

In [41]:
dataset.iloc[18]

title                                                             님그림자
lyrics               저만치 앞서가는 님 뒤로\n그림자 길게 드린 밤\n님의 그림자 밟으려하니\n서러움이...
genre                                                         성인가요/트로트
mungchi              [[3, 4, 1, 2], [3, 2, 2, 1], [2, 3, 5], [4, 3,...
label                [[저만치, 앞서가는, 님, 뒤로], [그림자, 길게, 드린, 밤], [님의, 그림...
chunk_strategy                                                    word
sampling_strategy                                                  one
syllable_limit                                                       0
Name: 19, dtype: object

In [30]:
dataset['lyrics'] = dataset['lyrics'].str.replace('\n\n', '\n', regex=False)

In [7]:
print(combined_df[combined_df['title'] =='님그림자']['lyrics'].str.replace('\n\n', '\n', regex=False).tolist())

['저만치 앞서가는 님 뒤로\n그림자 길게 드린 밤\n님의 그림자 밟으려하니\n서러움이 가슴에 이네\n님은 나의 마음 헤아릴까\n별만 해듯거린 밤\n휘훵한 달빛아래 님 뒤로\n긴 그림자 밟을 날 없네\n저만치 앞서가는 님 뒤로\n그림자 길게 드린 밤\n님의 그림자 밟으려하니\n서러움이 가슴에 이네\n님은 나의 마음 헤아릴까\n별만 헤듯 거린 밤\n휘훵한 달빛아래 님 뒤로\n긴 그림자 밟을 날 없네']


In [6]:
combined_df['lyrics'].str.replace('\n\n', '\n', regex=False)

0        J 스치는 바람에 J 그대 모습 보이면\n난 오늘도 조용히 그댈 그리워하네\nJ 지...
1        나는 나는 갯바위\n당신은 나를 사랑하는 파도\n어느 고운 바람 불던날\n잔잔히 다...
2        별이 유난히도 밝은\n오늘 이 시간이 가면\n그대\n떠난다~는 말이\n나를 슬프~게...
3        당신의 고운 눈매에 할말을 잊었지만은\n냉정히 돌아선 무정한 사람은 눈물을 모르겠지...
4        그대의 모습 멀어져갔네\n쓸쓸한 기억 남기고\n낭만이 지워진 그 길로\n조용히 멀어...
                               ...                        
14606    Scratch that bring it back\n(Shook! Shook! Ha)...
14607    우린 이 음악을 빌려\n이 분위기를 빌려\n지금 이 공기 이건 마치\n7시간 비행 ...
14608    사랑이었다 별거 없던 내 하루에\n빛이 돼준 단한 사람\n나보다 나를 더 아껴 주던...
14609    바람에 날려 꽃이 지는 계절엔\n아직도 너의 손을 잡은 듯 그런 듯 해\n그때는 아...
14610    난 지금 널 향해 달려가고 있어\n숨이 턱까지 차올라 괜찮아\n잠시 후 널 마주할 ...
Name: lyrics, Length: 6090, dtype: object